### libs

In [ ]:
import dspy
import openai
from openai import OpenAI
import pandas as pd
import os
import json
import hashlib
from tqdm import tqdm
import chromadb
import weaviate
from dspy.retrieve.weaviate_rm import WeaviateRM

import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

import time

# SECRET KEY OPENAI
secretk = os.getenv("OPENAI_API_KEY")
#client = OpenAI(api_key=secretk)
openai.api_key = secretk
client = OpenAI(api_key=secretk)

import pandas as pd
import os

os.chdir(os.path.abspath(os.curdir))

In [2]:
#os.listdir()

In [3]:
import re

# Função para limpar texto
def clean_text(text):
    # Decodificar caracteres Unicode
    text = text.encode().decode('utf-8')
    # Remover caracteres indesejados específicos (exemplo: \n, \r, etc.)
    text = re.sub(r'[\n\r]', ' ', text)
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [4]:
#df = pd.read_excel('OportunidadesMelhoriaJuntada.xlsx', header=3)
df = pd.read_excel('mapa_indicadores_RAG_21062024.xlsx')

In [6]:
df_ = df.iloc[:, :]
df_

,MERCADO,VISÃO,SWOT,OBJETIVO FINANCEIRO,RESULTADOS CHAVE FINANCEIRO,OBJETIVO CLIENTES E MERCADO,RESULTADOS CHAVE CLIENTES E MERCADO,OBJETIVO PROCESSOS,RESULTADOS CHAVE PROCESSOS,OBJETIVO PESSOAS,RESULTADOS CHAVE PESSOAS,OKRs
0,Banco,"Transformar vidas, facilitando o crédito.","Forças: cultura organizacional forte, proximid...",O1: Crescer receita de forma exponencial​.,O1KR1: Aumentar a receita de R$X para R$Y. O1K...,"O2: Ampliar a oferta de crédito e serviços, tr...",O2KR1: Aumentar a base de pessoa física do set...,"O3: Otimizar o fluxo de trabalho, proporcionan...",O3KR1:​ Aumentar o percentual de entregas no p...,O4: Fortalecer a cultura organizacional basead...,O4KR1: Obter o selo GPTW (nota>70) até 07/24. ...,O1: Crescer receita de forma exponencial​. O1K...
1,Imobiliária,Ser reconhecida entre as 5 melhores construtor...,"Forças: CREDIBILIDADE, CAPACIDADE TÉCNICA E VE...","O1: Alcançar saúde financeira, otimizando cust...",O1KR1: Garantir fluxo de caixa positivo (plane...,O2: Transformar cada contato com o cliente em ...,O2KR1: Implantar novo modelo de gestão 5 Estre...,"O3: Assegurar custo, prazo e qualidade das obras.",O3KR1: Reduzir a quantidade de revisão de proj...,O4: Fortalecer o orgulho em ser parte da empresa.,O4KR1: Atingir a nota ≥80% na pesquisa de sati...,"O1: Alcançar saúde financeira, otimizando cust..."
2,Indústria de alimentos,Ser a melhor indústria de beneficiamento de ca...,"Forças: qualidade do produto, credibilidade da...",O1: Tornar a empresa independente financeirame...,O1KR1: Aumentar faturamento anual de R$X para ...,O2: Ampliar a presença e o reconhecimento da m...,O2KR1: Implementar dois novos canais de venda ...,O3: Ser a fábrica mais eficiente do segmento d...,O3KR1: Alcançar 10 pontos no WCM em 2023. O3KR...,O4: Transformar a vida dos nossos colaboradore...,O4KR1: Obter a certificação de GPTW até o fina...,O1: Tornar a empresa independente financeirame...
3,Tecnologia,Ser referência nacional em soluções de gestão ...,"Forças, regras de negócio bem consolidadas no ...",O1: Garantir a rentabilidade do negócio.,O1KR1: Crescer 25% do faturamento. O1KR2: Gara...,O2 : Tornar nossos clientes apaixonados pela m...,O2KR1: Atender tickets dentro do SLA: de 65% p...,O3: Ser excelente em tudo que faz.,O3KR1: Padronizar 4 processos e suas respectiv...,O4: Ser reconhecida como a melhor empresa para...,O4KR1: Implementar um programa de desenvolvime...,O1: Garantir a rentabilidade do negócio. O1KR1...
4,Agricultura,Ser referência global na produção de alimentos...,"Forças: credibilidade da marca, equipe técnica...","O1: Ser uma empresa lucrativa, rentável e sust...",O1KR1: Aumentar o faturamento em X%. O1KR2: Re...,O2: Estar presente em todas as cidades do Brasil.,O2KR1: Reduzir a taxa de transformação da marc...,O3: Ser a melhor empresa de melão e melancia d...,O3KR1: Aumentar a produtividade média do melã...,O4: Ser uma empresa na qual as pessoas se orgu...,O4KR1: Reduzir rotatividade de X para Y. O4KR2...,"O1: Ser uma empresa lucrativa, rentável e sust..."
5,Textil,Gerar conexões e prosperidade para os negócios...,"Força: executar importação, prezar pela qualid...",O1: Crescer de forma rentável.,O1KR1: Aumentar as receitas de R$X para R$Y em...,O2: Transformar cada ponto de contato em uma e...,O2KR1: Atingir 20% da taxa de conversão. O2KR2...,NaN,NaN,O4: Fortalecer a cultura de engajamento do ne...,O4KR1: Aumentar a satisfação dos colaboradores...,O1: Crescer de forma rentável. O1KR1: Aumentar...


In [7]:
df_.columns

Index(['MERCADO', 'VISÃO', 'SWOT', 'OBJETIVO FINANCEIRO',
       'RESULTADOS CHAVE FINANCEIRO', 'OBJETIVO CLIENTES E MERCADO',
       'RESULTADOS CHAVE CLIENTES E MERCADO', 'OBJETIVO PROCESSOS',
       'RESULTADOS CHAVE PROCESSOS', 'OBJETIVO PESSOAS',
       'RESULTADOS CHAVE PESSOAS', 'OKRs'],
      dtype='object')

In [8]:
renomear = {
    'OBJETIVO FINANCEIRO':'OBJETIVOFINANCEIRO','RESULTADOS CHAVE FINANCEIRO':'RESULTADOSCHAVEFINANCEIRO', 
    'OBJETIVO CLIENTES E MERCADO':'OBJETIVOCLIENTESEMERCADO', 'RESULTADOS CHAVE CLIENTES E MERCADO':'RESULTADOSCHAVECLIENTESEMERCADO', 
    'OBJETIVO PROCESSOS':'OBJETIVOPROCESSOS', 'RESULTADOS CHAVE PROCESSOS': 'RESULTADOSCHAVEPROCESSOS', 
    'OBJETIVO PESSOAS':'OBJETIVOPESSOAS', 'RESULTADOS CHAVE PESSOAS':'RESULTADOSCHAVEPESSOAS'
}

df_.rename(columns=renomear, inplace=True)

In [9]:
df_.to_json('mapa_indicadores_GPT_okrs.json')

In [10]:
with open('mapa_indicadores_GPT_okrs.json', 'r') as file:
    data = json.load(file)
df = pd.DataFrame(data)

In [30]:
weaviate_client = weaviate.Client("http://localhost:8081", 
                                  additional_headers = {"X-OpenAI-Api-Key": secretk} 
                                 )

schema_ = weaviate_client.schema.get()

# deletando a classe existente
weaviate_client.schema.delete_all()
print(f"Classe limpa: {weaviate_client.schema.get()}, está ativa -> {weaviate_client.is_ready()}")

Classe limpa: {'classes': []}, está ativa -> True


In [31]:
df.columns

Index(['MERCADO', 'VISÃO', 'SWOT', 'OBJETIVOFINANCEIRO',
       'RESULTADOSCHAVEFINANCEIRO', 'OBJETIVOCLIENTESEMERCADO',
       'RESULTADOSCHAVECLIENTESEMERCADO', 'OBJETIVOPROCESSOS',
       'RESULTADOSCHAVEPROCESSOS', 'OBJETIVOPESSOAS', 'RESULTADOSCHAVEPESSOAS',
       'OKRs'],
      dtype='object')

#### Classe

In [32]:
try:
    weaviate_client.schema.delete_class("OKRs")
except:
    print("Erro ao deletar classe!")
finally:
    print("Dados deletados da classe.")
    
minha_classe = {
    
    "class" : "OKRs", 
    "description":"Aqui seguem exemplos de análises SWOT que geram OKR's.",
    "invertedIndexConfig": {
        "cleanupIntervalSeconds": 60
      },
    
    
    # 'SWOT', 'OBJETIVO1', 'RESULTADOS-CHAVE1', 'OBJETIVO2', 'RESULTADOS-CHAVE2'
                "properties": [
                        {"name": "SWOT",
                         "dataType": ["text"],
                         "description":"Contém o texto base do documento com OPORTUNIDADES DE MELHORIA(SOLUÇÃO INVESTIMENTO GANHOS)", 
                        },
                    
                        {"name": "VISAO",
                         "dataType": ["string"],
                         "description":"Visao do mercado."
                         },
                    
                        {"name": "MERCADO",
                         "dataType": ["string"],
                         "description":"nicho mercadologico."
                         },
                    
                         {"name": "OBJETIVOFINANCEIRO",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito financeiro."
                         },
                        {"name": "RESULTADOSCHAVEFINANCEIRO",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo financeiro."
                             },                  
              
                         {"name": "OBJETIVOCLIENTESEMERCADO",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito de analisar os clientes e o mercado."
                         },
                        {"name": "RESULTADOSCHAVECLIENTESEMERCADO",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo de clientes e mercado."
                             },
                    
                    
                             {"name": "OBJETIVOPROCESSOS",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito de analisar os processos."
                         },
                        {"name": "RESULTADOSCHAVEPROCESSOS",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo de processos."
                             },
                    
                    
                     {"name": "OBJETIVOPESSOAS",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito de analisar pessoas."
                         },
                        {"name": "RESULTADOSCHAVEPESSOAS",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo pessoas."
                             },
                    
                        {"name": "OKRs",
                             "dataType": ["text"],
                             "description":"OKRs que são gerados a partir da VISAO, MERCADO, OBJETIVOS e RESULTADOS."
                             } 
                  ],
    
    "vectorizer": "text2vec-openai",
    
    # gpt-3.5-turbo | gpt-3.5-turbo-16k | gpt-3.5-turbo-1106 | gpt-4 | gpt-4-32k | gpt-4-1106-preview
    
    "moduleConfig": {
        "generative-openai": {
          "model": "gpt-3.5-turbo-16k",
          "temperatureProperty": 0.7,  
          "maxTokensProperty":500,  
          #"frequencyPenaltyProperty": <frequency_penalty>,  // Optional, applicable to both OpenAI and Azure OpenAI
          #"presencePenaltyProperty": <presence_penalty>,  // Optional, applicable to both OpenAI and Azure OpenAI
          "topPProperty": 3,
        },
        
        "text2vec-openai": {
          "model": "text-embedding-3-large",
          "dimensions": 3072,
          "type": "text"          
        }
        
      },
    
    # Configure the vector index
    "vectorIndexType": "hnsw",
    "vectorIndexConfig": {
        "distance": "cosine",
        "pq": {
            "enabled": True,
            "segments": 192
        },
    },
    # Configure the inverted index
    "indexTimestamps": True,
    "indexNullState": True,
    "indexPropertyLength": True,

}


weaviate_client.schema.create_class(minha_classe)
print("Nova classe criada!")   

Dados deletados da classe.
Nova classe criada!


### Tratamento de dados e BATCH ADD

In [33]:
df.columns

Index(['MERCADO', 'VISÃO', 'SWOT', 'OBJETIVOFINANCEIRO',
       'RESULTADOSCHAVEFINANCEIRO', 'OBJETIVOCLIENTESEMERCADO',
       'RESULTADOSCHAVECLIENTESEMERCADO', 'OBJETIVOPROCESSOS',
       'RESULTADOSCHAVEPROCESSOS', 'OBJETIVOPESSOAS', 'RESULTADOSCHAVEPESSOAS',
       'OKRs'],
      dtype='object')

In [34]:
df.reset_index(drop=True, level=0, inplace=True)

In [35]:
df = df.fillna(" ")

In [36]:
df.columns

Index(['MERCADO', 'VISÃO', 'SWOT', 'OBJETIVOFINANCEIRO',
       'RESULTADOSCHAVEFINANCEIRO', 'OBJETIVOCLIENTESEMERCADO',
       'RESULTADOSCHAVECLIENTESEMERCADO', 'OBJETIVOPROCESSOS',
       'RESULTADOSCHAVEPROCESSOS', 'OBJETIVOPESSOAS', 'RESULTADOSCHAVEPESSOAS',
       'OKRs'],
      dtype='object')

In [37]:
lista = [] 
for i in range(len(df)):
    lista.append({
                    'MERCADO':df['MERCADO'][i],
                    'VISAO':df['VISÃO'][i],
                    'SWOT':df['SWOT'][i], 
                    'OBJETIVOFINANCEIRO':df['OBJETIVOFINANCEIRO'][i],
                    'RESULTADOSCHAVEFINANCEIRO':df['RESULTADOSCHAVEFINANCEIRO'][i], 
                    'OBJETIVOCLIENTESEMERCADO':df['OBJETIVOCLIENTESEMERCADO'][i],
                    'RESULTADOSCHAVECLIENTESEMERCADO':df['RESULTADOSCHAVECLIENTESEMERCADO'][i],
                    'OBJETIVOPROCESSOS':df['OBJETIVOPROCESSOS'][i],
                    'RESULTADOSCHAVEPROCESSOS':df['RESULTADOSCHAVEPROCESSOS'][i],
                    'OBJETIVOPESSOAS':df['OBJETIVOPESSOAS'][i],
                    'RESULTADOSCHAVEPESSOAS':df['RESULTADOSCHAVEPESSOAS'][i],
        
                    'OKRs':df['OKRs'][i]

                 })
    
lista

[{'MERCADO': 'Banco',
  'VISAO': 'Transformar vidas, facilitando o crédito.',
  'SWOT': 'Forças: cultura organizacional forte, proximidade com o cliente B2B, tecnologia própria, empresa regulamentada pelo BACEN, empresa escalável, dados disponíveis para gestão (BI), base para upsell e crossell, mindset de startup. Fraquezas: elaboração e execução de projetos, falta de processos integrados, gestão executando operacional, comunicação externa e interna, poucas automações de processos, alto custo operacional, falta de processos integrados, mix de produtos restrito para PJ e PF, falta de personalização do crédito (score), falta de foco na experiência do usuário. Oportunidades: falta de personalização do mercado para soluções financeiras e agilidade em fazer acontecer, carência na oferta de produtos e serviços financeiros direcionados para pequeno e médio PJ, restrição de crédito e alto nível de endividamento, transformação digital pós pandemia, grande quantidade de canais alternativos de ve

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   MERCADO                          6 non-null      object
 1   VISÃO                            6 non-null      object
 2   SWOT                             6 non-null      object
 3   OBJETIVOFINANCEIRO               6 non-null      object
 4   RESULTADOSCHAVEFINANCEIRO        6 non-null      object
 5   OBJETIVOCLIENTESEMERCADO         6 non-null      object
 6   RESULTADOSCHAVECLIENTESEMERCADO  6 non-null      object
 7   OBJETIVOPROCESSOS                6 non-null      object
 8   RESULTADOSCHAVEPROCESSOS         6 non-null      object
 9   OBJETIVOPESSOAS                  6 non-null      object
 10  RESULTADOSCHAVEPESSOAS           6 non-null      object
 11  OKRs                             6 non-null      object
dtypes: object(12)
memory usage: 708.0+ bytes

In [39]:
treino = [dspy.Example(question=f"""{df['SWOT'][i]}""", 
                       answer=df.loc[i, "OKRs"]).with_inputs("question") for i in range(len(df))]

In [40]:
import requests
import json

weaviate_client.batch.configure(batch_size=1000)  # Configure batch
with weaviate_client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(lista):  # Batch import data
        #print(f"importing question: {i + 1}")
        properties = {
                            'MERCADO':str(d['MERCADO']),
                            'VISAO':str(d['VISAO']),
                            'SWOT':str(d['SWOT']), 
                            'OBJETIVOFINANCEIRO':str(d['OBJETIVOFINANCEIRO']),
                            'RESULTADOSCHAVEFINANCEIRO':str(d['RESULTADOSCHAVEFINANCEIRO']), 
                            'OBJETIVOCLIENTESEMERCADO':str(d['OBJETIVOCLIENTESEMERCADO']),
                            'RESULTADOSCHAVECLIENTESEMERCADO':str(d['RESULTADOSCHAVECLIENTESEMERCADO']),
                            'OBJETIVOPROCESSOS':str(d['OBJETIVOPROCESSOS']),
                            'RESULTADOSCHAVEPROCESSOS':str(d['RESULTADOSCHAVEPROCESSOS']),
                            'OBJETIVOPESSOAS':str(d['OBJETIVOPESSOAS']),
                            'RESULTADOSCHAVEPESSOAS':str(d['RESULTADOSCHAVEPESSOAS']),
                            'OKRs':str(d['OKRs'])
                    }
        
        try:
            
            batch.add_data_object(
                data_object=properties,
                class_name="OKRs"
            )
        except InvalidJSONError as e:
            print(f'Erro {e}')

In [41]:
df_.columns

Index(['MERCADO', 'VISÃO', 'SWOT', 'OBJETIVOFINANCEIRO',
       'RESULTADOSCHAVEFINANCEIRO', 'OBJETIVOCLIENTESEMERCADO',
       'RESULTADOSCHAVECLIENTESEMERCADO', 'OBJETIVOPROCESSOS',
       'RESULTADOSCHAVEPROCESSOS', 'OBJETIVOPESSOAS', 'RESULTADOSCHAVEPESSOAS',
       'OKRs'],
      dtype='object')

### DSPy

In [42]:
_params4o = {"lm" : dspy.OpenAI(model='gpt-4o', max_tokens=1000, temperature=0.3, model_type='chat'), 
             "rm" : WeaviateRM("OKRs", weaviate_client = weaviate_client, 
                             # PASSAMOS O COLLECTION TEXT KEY QUE SERA USADO PARA CLASSIFICAR O TEXTO => (CONTEXT)
                             weaviate_collection_text_key="oKRs"
                            )
           }

_params35 = {"lm" : dspy.OpenAI(model='gpt-3.5-turbo-0125', max_tokens=1000, temperature=0.5, model_type='chat'), 
             "rm" : WeaviateRM("OKRs", weaviate_client = weaviate_client, 
                             # PASSAMOS O COLLECTION TEXT KEY QUE SERA USADO PARA CLASSIFICAR O TEXTO => (CONTEXT)
                             weaviate_collection_text_key="oKRs"
                            )
           }

# Passando kwargs
# avaliar as configs usando dspy.settings.config...
dspy.settings.configure(**_params4o)

### DSPy Oportune

In [43]:
class GeradorOKRs(dspy.Signature):
    
    """Você é um analista de projetos e processos de uma grande empresa de consultoria. 
       Sua tarefa é gerar OKRs para o cliente a partir da análise SWOT.
    """        
  
    # TEXTO DO PROBLEMA DE ENTRADA
    question = dspy.InputField()
    # AQUI TEMOS AS OPORTUNIDADES QUE VEM DA BASE FRIA (NO CASO ESTAMOS USANDO 5 PASSAGES)
    context = dspy.InputField(desc="""extraia informações da analise SWOT e dos objetivos e resultados-chave.""")    
   
    # RESPOSTA SEGUINDO OS FEWSHOTS dspy.Example()
    answer = dspy.OutputField(desc="""Dê sua resposta APENAS com as OKRs. Sugerir mais de 4 OKRs que façam sentido. Dê sua 
                                      resposta para cada perspectiva de okr: Financeira, Pessoas, Processos e Clientes.
                                      Pense passo a passo. RESPONDA em Português do Brasil.""")

class OKRsRAG(dspy.Module):
    def __init__(self, num_passages = 5): 
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GeradorOKRs)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)        
        return dspy.Prediction(answer=prediction.answer)

In [ ]:
from dspy.teleprompt import LabeledFewShot

labeled_fewshot_optimizer = LabeledFewShot(k=16)
modeloCompiladoOKR = labeled_fewshot_optimizer.compile(student = OKRsRAG(), trainset = treino)

In [ ]:
modeloCompiladoOKR

In [ ]:
modeloCompiladoOKR.save('modelo_oportune_21062024_okr.pkl')

### ENTRADA COM ANÁLISE SWOT E SAÍDA EM OKRs

In [ ]:
SWOT = """Forças:Variedade de Produtos: Amplia gama de pizzas, massas e acompanhamentos que atendem a diferentes gostos e preferências.
Localização Estratégica: Estabelecimento situado em uma área movimentada com fácil acesso para clientes, aumentando a visibilidade e conveniência.
Qualidade dos Ingredientes: Uso de ingredientes frescos e de alta qualidade, garantindo o sabor e a qualidade das pizzas.
Marca Reconhecida: Boa reputação local e base de clientes fiel, construída ao longo dos anos.
Fraquezas:Dependência de Mão-de-Obra: Setor de alimentação que depende muito de mão-de-obra, o que pode afetar a consistência e a eficiência do serviço.
Flutuações no Custo de Ingredientes: Variações nos preços dos ingredientes podem impactar os custos de produção e a lucratividade.
Concorrência Local: Presença de outras pizzarias na área competindo por clientes e mercado.
Oportunidades:Expansão do Delivery: Aumento na demanda por serviços de entrega de alimentos, oferecendo oportunidades para expandir o serviço de entrega.
Personalização e Inovação no Cardápio: Introdução de novos sabores de pizza, opções de massas especiais e promoções sazonais para atrair novos clientes.
Parcerias com Empresas Locais: Colaboração com empresas locais para fornecer catering em eventos ou promoções conjuntas para atrair novos clientes.
Ameaças:Concorrência de Cadeias de Fast-Food: Competição de grandes cadeias de fast-food que oferecem opções de refeições rápidas e baratas.
Flutuações Econômicas: Instabilidade econômica que pode afetar o poder de compra dos clientes e suas decisões de gasto.
Regulamentações de Saúde e Segurança: Requisitos e regulamentações governamentais que impactam as operações e custos de conformidade.
"""
#modelo = OportuneRAG()
okrs = modeloCompiladoOKR(question = SWOT)

#### OKRs de saída

In [ ]:
print(okrs.answer)

In [ ]:
_params4o['lm'].inspect_history(n=5)

### DSPy & Weaviate Cloud

In [ ]:
apikey = "yBpK1JBlsGGiYuxEBQnLRrIJNSdQSRVWhhhO"

In [3]:
import weaviate

auth_config = weaviate.AuthApiKey(api_key="yBpK1JBlsGGiYuxEBQnLRrIJNSdQSRVWhhhO")

wvc_client = weaviate.Client(
  url="https://db-vetores-okrs-qsklbzec.weaviate.network", 
  additional_headers = {"X-OpenAI-Api-Key": secretk}, 
  auth_client_secret=auth_config
)


C:\Users\ThiagoBluhm\.conda\envs\NLP_py311\Lib\site-packages\weaviate\__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
C:\Users\ThiagoBluhm\.conda\envs\NLP_py311\Lib\site-packages\weaviate\warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [ ]:
schema_ = wvc_client.schema.get()

# deletando a classe existente
#wvc_client.schema.delete_all()
#print(f"Classe limpa: {wvc_client.schema.get()}, está ativa -> {wvc_client.is_ready()}")

In [ ]:
try:
    wvc_client.schema.delete_class("OKRs")
except:
    print("Erro ao deletar classe!")
finally:
    print("Dados deletados da classe.")
    
minha_classe = {
    
    "class" : "OKRs", 
    "description":"Aqui seguem exemplos de análises SWOT que geram OKR's.",
    "invertedIndexConfig": {
        "cleanupIntervalSeconds": 60
      },
    
    
    # 'SWOT', 'OBJETIVO1', 'RESULTADOS-CHAVE1', 'OBJETIVO2', 'RESULTADOS-CHAVE2'
                "properties": [
                        {"name": "SWOT",
                         "dataType": ["text"],
                         "description":"Contém o texto base do documento com OPORTUNIDADES DE MELHORIA(SOLUÇÃO INVESTIMENTO GANHOS)", 
                        },
                    
                        {"name": "VISAO",
                         "dataType": ["string"],
                         "description":"Visao do mercado."
                         },
                    
                        {"name": "MERCADO",
                         "dataType": ["string"],
                         "description":"nicho mercadologico."
                         },
                    
                         {"name": "OBJETIVOFINANCEIRO",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito financeiro."
                         },
                        {"name": "RESULTADOSCHAVEFINANCEIRO",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo financeiro."
                             },                  
              
                         {"name": "OBJETIVOCLIENTESEMERCADO",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito de analisar os clientes e o mercado."
                         },
                        {"name": "RESULTADOSCHAVECLIENTESEMERCADO",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo de clientes e mercado."
                             },
                    
                    
                             {"name": "OBJETIVOPROCESSOS",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito de analisar os processos."
                         },
                        {"name": "RESULTADOSCHAVEPROCESSOS",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo de processos."
                             },
                    
                    
                     {"name": "OBJETIVOPESSOAS",
                         "dataType": ["string"],
                         "description":"objetivos implementados de acordo com a análise SWOT com intuito de analisar pessoas."
                         },
                        {"name": "RESULTADOSCHAVEPESSOAS",
                             "dataType": ["text"],
                             "description":"resultados chave para o objetivo pessoas."
                             },
                    
                        {"name": "OKRs",
                             "dataType": ["text"],
                             "description":"OKRs que são gerados a partir da VISAO, MERCADO, OBJETIVOS e RESULTADOS."
                             } 
                  ],
    
    "vectorizer": "text2vec-openai",
    
    # gpt-3.5-turbo | gpt-3.5-turbo-16k | gpt-3.5-turbo-1106 | gpt-4 | gpt-4-32k | gpt-4-1106-preview
    
    "moduleConfig": {
        "generative-openai": {
          "model": "gpt-3.5-turbo-16k",
          "temperatureProperty": 0.7,  
          "maxTokensProperty":500,  
          #"frequencyPenaltyProperty": <frequency_penalty>,  // Optional, applicable to both OpenAI and Azure OpenAI
          #"presencePenaltyProperty": <presence_penalty>,  // Optional, applicable to both OpenAI and Azure OpenAI
          "topPProperty": 3,
        },
        
        "text2vec-openai": {
          "model": "text-embedding-3-large",
          "dimensions": 3072,
          "type": "text"          
        }
        
      },
    
    # Configure the vector index
    "vectorIndexType": "hnsw",
    "vectorIndexConfig": {
        "distance": "cosine",
        "pq": {
            "enabled": True,
            "segments": 192
        },
    },
    # Configure the inverted index
    "indexTimestamps": True,
    "indexNullState": True,
    "indexPropertyLength": True,

}


wvc_client.schema.create_class(minha_classe)
print("Nova classe criada!")   

In [ ]:
import requests
import json

wvc_client.batch.configure(batch_size=1000)  # Configure batch
with wvc_client.batch as batch:  # Initialize a batch process
    for i, d in enumerate(lista):  # Batch import data
        #print(f"importing question: {i + 1}")
        properties = {
                            'MERCADO':str(d['MERCADO']),
                            'VISAO':str(d['VISAO']),
                            'SWOT':str(d['SWOT']), 
                            'OBJETIVOFINANCEIRO':str(d['OBJETIVOFINANCEIRO']),
                            'RESULTADOSCHAVEFINANCEIRO':str(d['RESULTADOSCHAVEFINANCEIRO']), 
                            'OBJETIVOCLIENTESEMERCADO':str(d['OBJETIVOCLIENTESEMERCADO']),
                            'RESULTADOSCHAVECLIENTESEMERCADO':str(d['RESULTADOSCHAVECLIENTESEMERCADO']),
                            'OBJETIVOPROCESSOS':str(d['OBJETIVOPROCESSOS']),
                            'RESULTADOSCHAVEPROCESSOS':str(d['RESULTADOSCHAVEPROCESSOS']),
                            'OBJETIVOPESSOAS':str(d['OBJETIVOPESSOAS']),
                            'RESULTADOSCHAVEPESSOAS':str(d['RESULTADOSCHAVEPESSOAS']),
                            'OKRs':str(d['OKRs'])
                    }
        
        try:
            
            batch.add_data_object(
                data_object=properties,
                class_name="OKRs"
            )
        except InvalidJSONError as e:
            print(f'Erro {e}')

In [4]:
_params4o = {"lm" : dspy.OpenAI(model='gpt-4o', max_tokens=1000, temperature=0.3, model_type='chat'), 
             "rm" : WeaviateRM("OKRs", weaviate_client = wvc_client, 
                             # PASSAMOS O COLLECTION TEXT KEY QUE SERA USADO PARA CLASSIFICAR O TEXTO => (CONTEXT)
                             weaviate_collection_text_key="oKRs"
                            )
           }

# Passando kwargs
# avaliar as configs usando dspy.settings.config...
dspy.settings.configure(**_params4o)

### DSPy Oportune

In [57]:
class GeradorOKRs(dspy.Signature):
    
    """Você é um analista de projetos e processos de uma grande empresa de consultoria. 
       Sua tarefa é gerar OKRs para o cliente a partir da análise SWOT.
    """        
  
    # TEXTO DO PROBLEMA DE ENTRADA
    question = dspy.InputField()
    # AQUI TEMOS AS OPORTUNIDADES QUE VEM DA BASE FRIA (NO CASO ESTAMOS USANDO 5 PASSAGES)
    context = dspy.InputField(desc="""extraia informações da analise SWOT e dos objetivos e resultados-chave.""")    
   
    # RESPOSTA SEGUINDO OS FEWSHOTS dspy.Example()
    answer = dspy.OutputField(desc="""Dê sua resposta APENAS com as OKRs. IMPORTANTE SUGERIR 4 ou mais OKRs PARA CADA UMA das 
                                      perspectivas de okr que são: Financeira, Pessoas, Processos e Clientes.
                                      Pense passo a passo. RESPONDA em Português do Brasil.""")

class OKRsRAG(dspy.Module):
    def __init__(self, num_passages = 5): 
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GeradorOKRs)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)        
        return dspy.Prediction(answer=prediction.answer)

In [60]:
from dspy.teleprompt import LabeledFewShot

labeled_fewshot_optimizer = LabeledFewShot(k=16)
modeloCompiladoOKR = labeled_fewshot_optimizer.compile(student = OKRsRAG(), trainset = treino)

In [62]:
modeloCompiladoOKR.save('modelo_oportune_29072024_okrs.pkl')

### ENTRADA COM ANÁLISE SWOT E SAÍDA EM OKRs

In [63]:
SWOT = """Forças:Variedade de Produtos: Amplia gama de pizzas, massas e acompanhamentos que atendem a diferentes gostos e preferências.
Localização Estratégica: Estabelecimento situado em uma área movimentada com fácil acesso para clientes, aumentando a visibilidade e conveniência.
Qualidade dos Ingredientes: Uso de ingredientes frescos e de alta qualidade, garantindo o sabor e a qualidade das pizzas.
Marca Reconhecida: Boa reputação local e base de clientes fiel, construída ao longo dos anos.
Fraquezas:Dependência de Mão-de-Obra: Setor de alimentação que depende muito de mão-de-obra, o que pode afetar a consistência e a eficiência do serviço.
Flutuações no Custo de Ingredientes: Variações nos preços dos ingredientes podem impactar os custos de produção e a lucratividade.
Concorrência Local: Presença de outras pizzarias na área competindo por clientes e mercado.
Oportunidades:Expansão do Delivery: Aumento na demanda por serviços de entrega de alimentos, oferecendo oportunidades para expandir o serviço de entrega.
Personalização e Inovação no Cardápio: Introdução de novos sabores de pizza, opções de massas especiais e promoções sazonais para atrair novos clientes.
Parcerias com Empresas Locais: Colaboração com empresas locais para fornecer catering em eventos ou promoções conjuntas para atrair novos clientes.
Ameaças:Concorrência de Cadeias de Fast-Food: Competição de grandes cadeias de fast-food que oferecem opções de refeições rápidas e baratas.
Flutuações Econômicas: Instabilidade econômica que pode afetar o poder de compra dos clientes e suas decisões de gasto.
Regulamentações de Saúde e Segurança: Requisitos e regulamentações governamentais que impactam as operações e custos de conformidade.
"""
#modelo = OportuneRAG()
#okrs = modeloCompiladoOKR(question = SWOT)

In [64]:
modelo = OKRsRAG()
modelo.load('modelo_oportune_29072024_okrs.pkl') #(f'modelo_oportune_23062024_okr_cloud.pkl')

In [65]:
okrs = modelo(question=SWOT)

#### OKRs de saída

In [67]:
print(okrs.answer)

**Financeira:**
1. **O1: Aumentar a rentabilidade do negócio.**
   - O1KR1: Aumentar o faturamento em 20% até o final de 2023.
   - O1KR2: Reduzir os custos operacionais em 15% até o final de 2023.
   - O1KR3: Aumentar a margem de lucro líquido de 10% para 15% até o final de 2023.
   - O1KR4: Implementar um sistema de gestão de custos para monitorar flutuações no preço dos ingredientes até junho de 2023.

**Clientes:**
2. **O2: Melhorar a experiência do cliente e aumentar a fidelização.**
   - O2KR1: Aumentar a taxa de satisfação do cliente (NPS) de 70 para 85 até dezembro de 2023.
   - O2KR2: Expandir o serviço de delivery para cobrir 100% da área urbana até setembro de 2023.
   - O2KR3: Introduzir 5 novos sabores de pizza e 3 novas opções de massas até julho de 2023.
   - O2KR4: Implementar um programa de fidelidade para clientes até março de 2023.

**Processos:**
3. **O3: Otimizar os processos internos para aumentar a eficiência.**
   - O3KR1: Reduzir o tempo de preparação e entrega

In [ ]:
_params4o['lm'].inspect_history(n=5)